# Merced land classification

Some default settings.

In [1]:
%config InlineBackend.figure_format='retina'
%matplotlib inline

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
from sklearn.utils import check_random_state


First, we convert all .tif images to .png for convenience (ImageDatGenerator only accepts .png or .jpeg)

In [2]:
import os, glob, subprocess
from PIL import Image

#Convert all input images from .tif to .png
thispath = os.getcwd()
cmd=str( 'mkdir %s/InputFiles'%thispath )
os.system(cmd)
cats = os.listdir( '%s/UCMerced_LandUse/Images/'%thispath )
for cat in cats:
    newpath=str( '%s/InputFiles/%s'%(thispath,cat) )
    cmd=str( 'mkdir -p %s'%(newpath) )
    os.system(cmd)
    fullpath = str( "%s/UCMerced_LandUse/Images/%s"%(thispath,cat) ) 
    images = glob.glob("%s/*.tif"%fullpath)
    for infile in images:
        f, e = os.path.splitext(infile)
        outfile = newpath + "/" + f.split("/")[-1] + ".png"
        if infile != outfile:
            im = Image.open(infile)
            im.save(outfile, format = "png")

Now generate 10 new images for every image in the dataset ("data augmentation")

In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=180,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            channel_shift_range=0.1,
            fill_mode='nearest')

#Make 10 "random" copies for each image of each category.
for cat in cats:
    directory = str( '%s/InputFiles/%s'%(thispath,cat) )
    print('Transforming files in %s'%directory)
    files=os.listdir( directory )
    filenum=0
    for file in files:
        img = load_img( directory + '/' + file )
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape)  
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=directory, save_prefix=cat+'_new_'+str(filenum), save_format='png'):
            i += 1
            if i >= 10:
                break
        filenum += 1

Using Theano backend.


Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/agricultural
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/airplane
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/baseballdiamond
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/beach
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/buildings
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/chaparral
Transforming files in /Users/Vincenzo/Desktop/EPFL/AdvancedComputingTechniques/Exercises/advanced-comp-2017/FinalProject/InputFiles/denseresiden